<a href="https://colab.research.google.com/github/dsevero/generative-models/blob/master/experiments/variational-inference/notebooks/Variational_Lower_Bound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>